In [1]:
import os
import sys
import os
os.chdir('/scratch/nhl256/dl_project/code/')
from PIL import Image

import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import *
from nhung_data_helper import FastRCNNLabeledDataset
from helper import *


import math
import pickle
import time
import copy

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### 05/02/20
- Preprocess 6 images before passing through the model
- Saving the feature extractor and the fasterRCNN model

  

In [3]:
# All the images are saved in image_folder
# All the labels are saved in the annotation_csv file

# image_folder = '../data'
# annotation_csv = '../data/annotation.csv'

image_folder = 'data/data'
annotation_csv = 'data/data/annotation.csv'


cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
print(device)

# image_folder = '/Users/nhungle/Downloads/dl20_data'
# annotation_csv = '/Users/nhungle/Downloads/dl20_data/annotation.csv'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
print(cuda)

True


In [ ]:
labeled_scene_index = np.arange(106, 134)
random.seed(1008)
random.shuffle(labeled_scene_index)
train_labeled_scene_index = labeled_scene_index[:22]
val_labeled_scene_index = labeled_scene_index[22:26]
test_labeled_scene_index = labeled_scene_index[26:]
BATCH_SIZE=1


transform = torchvision.transforms.ToTensor()
fasterRCNN_trainset = FastRCNNLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=train_labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
train_loader = torch.utils.data.DataLoader(fasterRCNN_trainset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2, collate_fn=collate_fn)


fasterRCNN_valset = FastRCNNLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=val_labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
val_loader = torch.utils.data.DataLoader(fasterRCNN_valset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2, collate_fn=collate_fn)


fasterRCNN_testset = FastRCNNLabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=test_labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
test_loader = torch.utils.data.DataLoader(fasterRCNN_testset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2, collate_fn=collate_fn)

In [60]:
train_loader.__len__()

126

### Get Feature Extractor

In [ ]:

feature_extractor = torchvision.models.resnet18(pretrained=False)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
feature_extractor.to(device)
for param in feature_extractor.parameters():
    param.requires_grad = True

In [ ]:
def concat_features(features, dim = 2):
    #dim 0 ==> stacking the images in the channel dimension
    #dim 1 ==> stacking the images in row dimension
    #dim 2 ==> stacking the images in column dimension
    tensor_tuples = torch.unbind(features, dim=0)
    concatenated_fm = torch.cat(tensor_tuples, dim=dim)
    return concatenated_fm 

### Inspect if it works on a pretrained FasterRCNN

In [67]:
sample, targets = next(iter(train_loader))
sample = torch.stack(sample)
sample = sample.to(device)
batchsize = sample.shape[0]
fe_batch = []
for i in range(batchsize):
    image_tensor = sample[i]
    features = feature_extractor(image_tensor)
    #print(features.shape)
    features = concat_features(features)
    features = features.view(3, 512, 160)
    #print(features.shape)
    fe_batch.append(features)

images = list(image.to(device) for image in fe_batch)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

torch.Size([1, 6, 3, 256, 306])

In [74]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)
output1 = model(images, targets)
output1

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


{'loss_box_reg': tensor(0.0014, device='cuda:0', grad_fn=<DivBackward0>),
 'loss_classifier': tensor(0.1285, device='cuda:0', grad_fn=<NllLossBackward>),
 'loss_objectness': tensor(1.0108, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(0.8967, device='cuda:0', grad_fn=<DivBackward0>)}

## Train One Epoch

In [ ]:
import math
import sys
import time
import torch

import utils

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 1
epoch = 0
print_freq = 20

In [ ]:
def train_one_epoch(feature_extractor, model, optimizer, data_loader, device, epoch, print_freq):
    feature_extractor.train()
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for sample, targets in metric_logger.log_every(data_loader, print_freq, header):
        sample = torch.stack(sample)
        sample = sample.to(device)
        batchsize = sample.shape[0]
        fe_batch = []
        for i in range(batchsize):
            image_tensor = sample[i]
            features = feature_extractor(image_tensor)
            #print(features.shape)
            features = concat_features(features)
            features = features.view(3, 512, 160)
            #print(features.shape)
            fe_batch.append(features)

        images = list(image.to(device) for image in fe_batch)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        #print(loss_dict)

        losses = sum(loss for loss in loss_dict.values())
        #print(losses)

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return losses

### Inspect train_one_epoch

In [112]:
train_one_epoch(feature_extractor, model, optimizer, test_loader, device, epoch, print_freq)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Epoch: [0]  [  0/126]  eta: 0:00:45  lr: 0.000045  loss: 3.8206 (3.8206)  loss_classifier: 0.1427 (0.1427)  loss_box_reg: 0.0051 (0.0051)  loss_objectness: 2.2652 (2.2652)  loss_rpn_box_reg: 1.4076 (1.4076)  time: 0.3627  data: 0.1629  max mem: 8165
Epoch: [0]  [ 20/126]  eta: 0:00:19  lr: 0.000844  loss: 2.0000 (2.2709)  loss_classifier: 0.1150 (0.1568)  loss_box_reg: 0.0036 (0.0054)  loss_objectness: 0.5195 (0.9075)  loss_rpn_box_reg: 1.0376 (1.2011)  time: 0.1788  data: 0.0030  max mem: 8432
Epoch: [0]  [ 40/126]  eta: 0:00:15  lr: 0.001643  loss: 1.3916 (1.9128)  loss_classifier: 0.1019 (0.1338)  loss_box_reg: 0.0013 (0.0037)  loss_objectness: 0.3839 (0.6369)  loss_rpn_box_reg: 0.9212 (1.1384)  time: 0.1789  data: 0.0033  max mem: 8432
Epoch: [0]  [ 60/126]  eta: 0:00:11  lr: 0.002443  loss: 1.0471 (1.6605)  loss_classifier: 0.0957 (0.1286)  loss_box_reg: 0.0022 (0.0042)  loss_objectness: 0.1140 (0.4670)  loss_rpn_box_reg: 0.7875 (1.0607)  time: 0.1782  data: 0.0032  max mem: 8432


tensor(1.9401, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
sample, targets = next(iter(train_loader))
sample = torch.stack(sample)
sample = sample.to(device)
batchsize = sample.shape[0]
fe_batch = []
for i in range(batchsize):
    image_tensor = sample[i]
    features = feature_extractor(image_tensor)
    #print(features.shape)
    features = concat_features(features)
    features = features.view(3, 512, 160)
    #print(features.shape)
    fe_batch.append(features)

images = list(image.to(device) for image in fe_batch)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [119]:
model.eval()
predictions = model(images)
print(predictions)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


In [ ]:
# model_test = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# model_test = model_test.to(device)
# model_test.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# x = list(image.to(device) for image in x)
# predictions = model_test(x)
# predictions

## Evaluate One Epoch

In [ ]:
sample, targets = next(iter(val_loader))
sample = torch.stack(sample)
sample = sample.to(device)

In [ ]:
def reorder_coord(pred_bboxes):
    xmin, ymin, xmax, ymax = pred_bboxes.unbind(1)
    return torch.stack((xmax, xmax, xmin, xmin, ymax, ymin, ymax, ymin), dim=1).view(-1, 2, 4)

def get_bounding_boxes(samples):
    # samples is a cuda tensor with size [batch_size, 6, 3, 256, 306]
    # You need to return a tuple with size 'batch_size' and each element is a cuda tensor [N, 2, 4]
    # where N is the number of object

    #Preparing inputs
    batchsize = samples.shape[0]
    fe_batch = []
    for i in range(batchsize):
        image_tensor = sample[i]
        features = feature_extractor(image_tensor)
        #print(features.shape)
        features = concat_features(features)
        features = features.view(3, 512, 160)
        #print(features.shape)
        fe_batch.append(features)

    images = list(image.to(device) for image in fe_batch)
    predictions = model(images)
    res = []
    for i in range(len(predictions)):
        prediction = predictions[i]
        pred_bboxes = prediction['boxes']
        reorder_pred_bboxes = reorder_coord(pred_bboxes)
        res.append(reorder_pred_bboxes)

    return res

In [ ]:
def eval_one_epoch(feature_extractor, model, dataloader):
    model.eval()
    feature_extractor.eval()
    total = 0
    total_ats_bounding_boxes = 0
    for i, data in enumerate(dataloader):
        total += 1
        sample, target = data
        sample = torch.stack(sample)
        sample = sample.cuda()

        predicted_bounding_boxes = get_bounding_boxes(sample)[0].cpu()
        

        ats_bounding_boxes = compute_ats_bounding_boxes(predicted_bounding_boxes,
                                                        target[0]['bounding_box'])
        print('Number of pred bboxes {}'.format(predicted_bounding_boxes.shape))
        print('ats_bounding_boxes {}'.format(ats_bounding_boxes))

        total_ats_bounding_boxes += ats_bounding_boxes
    return total_ats_bounding_boxes

In [ ]:
total_ats_bounding_boxes = eval_one_epoch(feature_extractor, model, test_loader)


## Train and Eval for multiple epochs

In [ ]:
# Get model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)

# Get feature extractor
feature_extractor = torchvision.models.resnet18(pretrained=False)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2])
feature_extractor.to(device)
for param in feature_extractor.parameters():
    param.requires_grad = True


# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 1
epoch = 0
print_freq = 500

In [ ]:
def train_val(feature_extractor, model, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
    best_model_wts = best_model_wts = {'feature_extractor': copy.deepcopy(feature_extractor.state_dict()),
                                       'fasterRCNN': copy.deepcopy(model.state_dict())
                                      }
    losses = []
    total_ats = []
    best_total_ats = -1

    loss = train_one_epoch(feature_extractor, model, optimizer, train_loader, device, epoch, print_freq)
    total_ats_bounding_boxes = eval_one_epoch(feature_extractor, model, val_loader)
    losses.append(loss)
    total_ats.append(total_ats_bounding_boxes)
    print('epoch {} loss {} total_ats {}'.format(epoch, loss, total_ats))

    if total_ats_bounding_boxes > best_total_ats:
        best_total_ats = total_ats_bounding_boxes
        best_model_wts = {'feature_extractor': copy.deepcopy(feature_extractor.state_dict()),
                           'fasterRCNN': copy.deepcopy(model.state_dict())
                                 }
        torch.save(best_model_wts, '/scratch/nhl256/dl_project/model/object_detection_resnet18_0502_epoch{}.pth'.format(epoch))

    return losses, total_ats, best_model_wts

In [122]:
import pickle
pickle.dump(train_losses, open('/scratch/nhl256/dl_project/model/object_detection_resnet18_0502_trainlosses.pickle', "wb"))
pickle.dump(eval_total_ats, open('/scratch/nhl256/dl_project/model/object_detection_resnet18_0502_evalTotalAts.pickle', "wb"))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Epoch: [0]  [  0/126]  eta: 0:00:47  lr: 0.000045  loss: 3.5512 (3.5512)  loss_classifier: 0.2599 (0.2599)  loss_box_reg: 0.0063 (0.0063)  loss_objectness: 1.8867 (1.8867)  loss_rpn_box_reg: 1.3983 (1.3983)  time: 0.3789  data: 0.1737  max mem: 10063
Epoch: [0]  [ 20/126]  eta: 0:00:19  lr: 0.000844  loss: 1.8488 (2.2739)  loss_classifier: 0.1346 (0.1544)  loss_box_reg: 0.0044 (0.0080)  loss_objectness: 0.5872 (0.9771)  loss_rpn_box_reg: 1.0572 (1.1344)  time: 0.1791  data: 0.0035  max mem: 10371
Epoch: [0]  [ 40/126]  eta: 0:00:15  lr: 0.001643  loss: 1.3144 (1.7959)  loss_classifier: 0.1007 (0.1331)  loss_box_reg: 0.0015 (0.0057)  loss_objectness: 0.2158 (0.6163)  loss_rpn_box_reg: 0.9558 (1.0409)  time: 0.1784  data: 0.0033  max mem: 10371
Epoch: [0]  [ 60/126]  eta: 0:00:12  lr: 0.002443  loss: 1.1813 (1.5778)  loss_classifier: 0.0880 (0.1252)  loss_box_reg: 0.0017 (0.0051)  loss_objectness: 0.1126 (0.4593)  loss_rpn_box_reg: 0.9283 (0.9881)  time: 0.1786  data: 0.0033  max mem: 10

NameError: ignored

## Evaluate - IoU by Class

In [ ]:
def prepare_pred_results(predictions):
    pred_boxes = []
    pred_labels = []
    pred_scores = []
    for prediction in predictions:
        #print(prediction)
        if len(prediction) == 0:
            continue
        boxes = prediction["boxes"]
        boxes = reorder_coord(boxes).tolist()
        scores = prediction["scores"].tolist()
        labels = prediction["labels"].tolist()

        pred_boxes.append(boxes)
        pred_labels.append(labels)
        pred_scores.append(scores)

    return pred_boxes, pred_labels, pred_scores

def reorder_coord(boxes):
    xmin, ymin, xmax, ymax = boxes.unbind(1)
    return torch.stack((ymin, xmin, ymax, xmax), dim=1)

def prepare_gt(targets):
    gt_boxes = []
    gt_labels = []
    for target in targets:
        boxes = target['boxes']
        boxes = reorder_coord(boxes).tolist()
        labels = target["labels"].tolist()
        gt_boxes.append(boxes)
        gt_labels.append(labels)
    return gt_boxes, gt_labels

In [ ]:
# Make sure that bbox_a, bbox_b = np.array

def bbox_iou(bbox_a, bbox_b):
    #print(type(bbox_a), type(bbox_b))
    bbox_a = np.array(bbox_a)
    bbox_b = np.array(bbox_b)

    # print(type(bbox_a), type(bbox_b))
    # print(bbox_a.shape, bbox_b.shape)
    if bbox_a.shape[1] != 4 or bbox_b.shape[1] != 4:
        raise IndexError

    # top left (i.e., ymin, xmin)
    tl = np.maximum(bbox_a[:, None, :2], bbox_b[:, :2])
    # bottom right (i.e., ymax, xmax)
    br = np.minimum(bbox_a[:, None, 2:], bbox_b[:, 2:])

    # Area of intersection: (tl < br) = bool, (br-tl) = (ymax-ymin) 
    area_i = np.prod(br - tl, axis=2) * (tl < br).all(axis=2)
    area_a = np.prod(bbox_a[:, 2:] - bbox_a[:, :2], axis=1)
    area_b = np.prod(bbox_b[:, 2:] - bbox_b[:, :2], axis=1)

    return area_i / (area_a[:, None] + area_b - area_i)

In [ ]:
def cal_TP_FP_iou(pred_bbox_c, gt_bbox_c, iou_thres=0.5):
    iou_table = bbox_iou(pred_bbox_c, gt_bbox_c)
    num_pred_bboxes = iou_table.shape[0]
    num_gt_bboxes = iou_table.shape[1]
    TP = np.zeros(num_pred_bboxes)
    FP = np.zeros(num_pred_bboxes)
    # For each pred_bounding box:
      # Find the most relevant gt_bbox (i.e., the gt_bbox with max IoU)
      # If IoU < threshold, then flag it as FP
      # If IoU >= threshold, then:
        # If that gt_bbox already has already matched with another pred_bbox:
          # Flag it as FP
        # Else:
          # Flag it as TP

    # TP only happens if the pred_bbox mathes with a gt_bbox
    for i in range(num_pred_bboxes):
        gt_bbox_index = np.argmax(iou_table[i])
        best_pred_bbox_index_for_selected_gt_bbox = np.argmax(iou_table[:,gt_bbox_index])
        if iou_table[i, gt_bbox_index] > iou_thres \
            and gt_bbox_index == best_pred_bbox_index_for_selected_gt_bbox:
            TP[i] = 1
        else:
            FP[i] = 1

    TP_cum = np.sum(TP)
    FP_cum = np.sum(FP)

    if (TP_cum + FP_cum) != num_pred_bboxes:
        print("WRONG CALCULATION OF FP")
    return TP_cum, FP_cum

In [ ]:
# Test for cal_TP_FP_iou

def inspect_call_TP_FP_iou(test_images, test_targets):
    test_images = torch.stack(test_images)
    #print(test_images.shape)
    test_images = prepare_inputs(test_images)
    #print(test_images[0].shape)

    test_images = list(image.to(device) for image in test_images)
    test_targets = [{k: v.to(device) for k, v in t.items()} for t in test_targets]

    model.eval()
    predictions = model(test_images)

    pred_bboxes, pred_labels, pred_scores = prepare_pred_results(predictions)
    gt_bboxes, gt_labels = prepare_gt(test_targets)

    for pred_bbox, pred_label, pred_score, gt_bbox, gt_label in \
        zip(pred_bboxes, pred_labels, pred_scores, gt_bboxes, gt_labels):
        pred_bbox = np.array(pred_bbox)
        pred_score = np.array(pred_score)
        pred_label = np.array(pred_label)
        gt_bbox = np.array(gt_bbox)
        gt_label = np.array(gt_label)
        unique_share_classes = (np.unique(np.concatenate((pred_label, gt_label))))
        
        for c in unique_share_classes:
            pred_class_c_index = np.where(pred_label == c)[0]
            pred_bbox_c = pred_bbox[pred_class_c_index]
            gt_class_c_index = np.where(gt_label == c)[0]
            #print(gt_class_c_index)
            gt_bbox_c = gt_bbox[gt_class_c_index]
            num_gt_bboxes = len(gt_class_c_index)
            num_pred_bboxes = len(pred_class_c_index)
            print('class {} with {} gt_bboxes and {} pred_bboxes'.format(c, num_gt_bboxes, num_pred_bboxes))
            # print(num_gt_bboxes)
            # print(num_pred_bboxes)
            if num_pred_bboxes == 0:
                class_TP = 0
                class_FP = 0
                class_FN = num_gt_bboxes
            elif num_gt_bboxes == 0:
                class_TP = 0
                class_FP = num_pred_bboxes
                class_FN = 0
            else:
                class_TP, class_FP = cal_TP_FP_iou(pred_bbox_c, gt_bbox_c, iou_thres)
                class_FN = num_gt_bboxes - class_TP
                print(class_TP + class_FP == num_pred_bboxes)

In [ ]:
# for i in range(3):
#     print('Iter {}'.format(i))
#     test_images, test_targets = next(iter(test_loader))
#     inspect_call_TP_FP_iou(test_images, test_targets)

In [ ]:
def evaluate_one_batch(predictions, test_targets, res, iou_thres=0.5):

    pred_bboxes, pred_labels, pred_scores = prepare_pred_results(predictions)
    gt_bboxes, gt_labels = prepare_gt(test_targets)
    # res stores the TP_FP dict for each class
    # Each TP_FP dict stores the TP_FP for each class 
    
    batch_total_TP = 0
    batch_total_FP = 0
    batch_total_FN = 0
    batch_total_num_object = 0
    batch_res = {c: {'TP':0, 'FP': 0, 'FN': 0} for c in range(9)}

    for pred_bbox, pred_label, pred_score, gt_bbox, gt_label in \
        zip(pred_bboxes, pred_labels, pred_scores, gt_bboxes, gt_labels):

        pred_bbox = np.array(pred_bbox)
        pred_score = np.array(pred_score)
        pred_label = np.array(pred_label)
        gt_bbox = np.array(gt_bbox)
        gt_label = np.array(gt_label)
        unique_share_classes = (np.unique(np.concatenate((pred_label, gt_label))))
        
        for c in unique_share_classes:
            pred_class_c_index = np.where(pred_label == c)[0]
            pred_bbox_c = pred_bbox[pred_class_c_index]
            gt_class_c_index = np.where(gt_label == c)[0]
            #print(gt_class_c_index)
            gt_bbox_c = gt_bbox[gt_class_c_index]
            num_gt_bboxes = len(gt_class_c_index)
            num_pred_bboxes = len(pred_class_c_index)
            #print('class {} with {} gt_bboxes and {} pred_bboxes'.format(c, num_gt_bboxes, num_pred_bboxes))
            if num_pred_bboxes == 0:
                class_TP = 0
                class_FP = 0
                class_FN = num_gt_bboxes
            elif num_gt_bboxes == 0:
                class_TP = 0
                class_FP = num_pred_bboxes
                class_FN = 0
            else:
                class_TP, class_FP = cal_TP_FP_iou(pred_bbox_c, gt_bbox_c, iou_thres)
                class_FN = num_gt_bboxes - class_TP
                #print(class_TP + class_FP == num_pred_bboxes)

            batch_total_TP += class_TP
            batch_total_FP += class_FP
            batch_total_FN += class_FN
            batch_total_num_object += num_gt_bboxes

            batch_res[c]['TP'] += class_TP
            batch_res[c]['FP'] += class_FP
            batch_res[c]['FN'] += class_FN

            res[c]['TP'] += class_TP
            res[c]['FP'] += class_FP
            res[c]['FN'] += class_FN
            
    return res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object

In [ ]:
# Inspect evaluate_one_batch
def inspect_evaluate_one_batch(test_images, test_targets, final_res):
    test_images = torch.stack(test_images)
    #print(test_images.shape)
    test_images = prepare_inputs(test_images)
    #print(test_images[0].shape)

    test_images = list(image.to(device) for image in test_images)
    test_targets = [{k: v.to(device) for k, v in t.items()} for t in test_targets]

    model.eval()
    predictions = model(test_images)

    final_res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object \
    = evaluate_one_batch(predictions, test_targets, final_res, iou_thres=0.5)

    return final_res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object

In [ ]:

final_res = {c: {'TP':0, 'FP': 0, 'FN': 0} for c in range(9)}
final_TP = 0
final_FP = 0
final_FN = 0
final_num_objects = 0

# test for 2 batches
for i in range(2):
    test_images, test_targets = next(iter(test_loader))
    final_res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object \
    = inspect_evaluate_one_batch(test_images, test_targets, final_res)

    print('batch {}'.format(i))

    
    print(batch_total_TP, batch_total_FN, batch_total_num_object)
    print('cur batch res:', batch_res)
    print('final res after this batch:', final_res)

batch 0
0 20 20
cur batch res: {0: {'TP': 0, 'FP': 0, 'FN': 1}, 1: {'TP': 0, 'FP': 0, 'FN': 0}, 2: {'TP': 0, 'FP': 0, 'FN': 16}, 3: {'TP': 0, 'FP': 0, 'FN': 2}, 4: {'TP': 0, 'FP': 0, 'FN': 1}, 5: {'TP': 0, 'FP': 0, 'FN': 0}, 6: {'TP': 0, 'FP': 0, 'FN': 0}, 7: {'TP': 0, 'FP': 0, 'FN': 0}, 8: {'TP': 0, 'FP': 0, 'FN': 0}}
final res after this batch: {0: {'TP': 0, 'FP': 0, 'FN': 1}, 1: {'TP': 0, 'FP': 0, 'FN': 0}, 2: {'TP': 0, 'FP': 0, 'FN': 16}, 3: {'TP': 0, 'FP': 0, 'FN': 2}, 4: {'TP': 0, 'FP': 0, 'FN': 1}, 5: {'TP': 0, 'FP': 0, 'FN': 0}, 6: {'TP': 0, 'FP': 0, 'FN': 0}, 7: {'TP': 0, 'FP': 0, 'FN': 0}, 8: {'TP': 0, 'FP': 0, 'FN': 0}}
batch 1
0 20 20
cur batch res: {0: {'TP': 0, 'FP': 0, 'FN': 1}, 1: {'TP': 0, 'FP': 0, 'FN': 0}, 2: {'TP': 0, 'FP': 0, 'FN': 19}, 3: {'TP': 0, 'FP': 0, 'FN': 0}, 4: {'TP': 0, 'FP': 0, 'FN': 0}, 5: {'TP': 0, 'FP': 0, 'FN': 0}, 6: {'TP': 0, 'FP': 0, 'FN': 0}, 7: {'TP': 0, 'FP': 0, 'FN': 0}, 8: {'TP': 0, 'FP': 0, 'FN': 0}}
final res after this batch: {0: {'TP': 0

In [ ]:
def evaluate_one_epoch(test_loader, iou_thres=0.5):
    # Evaluate. for all data point in the evaluaton set
    final_res = {c: {'TP':0, 'FP': 0, 'FN': 0} for c in range(9)}
    final_TP = 0
    final_FP = 0
    final_FN = 0
    final_num_objects = 0

    for iter_, (test_images, test_targets) in enumerate(test_loader):
        # if iter_ % 50 == 0:
        #     print('iter', iter_)
        #print('iter', iter_)
        test_images = torch.stack(test_images)
        #print(test_images.shape)
        test_images = prepare_inputs(test_images)
        #print(test_images[0].shape)

        test_images = list(image.to(device) for image in test_images)
        test_targets = [{k: v.to(device) for k, v in t.items()} for t in test_targets]

        model.eval()
        predictions = model(test_images)

        # Evaluate for one batch
        final_res, batch_res, batch_total_TP, batch_total_FP, batch_total_FN, batch_total_num_object \
                    = evaluate_one_batch(predictions, test_targets, final_res, iou_thres=0.5)

        
        final_TP += batch_total_TP
        final_FP += batch_total_FP
        final_FN += batch_total_FN
        final_num_objects += batch_total_num_object

    return final_res, final_TP, final_FP, final_FN, final_num_objects

In [ ]:
final_res, final_TP, final_FP, final_FN, final_num_objects = evaluate_one_epoch(test_loader, iou_thres=0.5)

iter 0
iter 50
iter 100


In [ ]:
def evaluate_threst_score(TP, FP, FN):
    return (TP / (TP + FP + FN))

In [ ]:
final_res

{0: {'FN': 158, 'FP': 0, 'TP': 0},
 1: {'FN': 0, 'FP': 0, 'TP': 0},
 2: {'FN': 3074, 'FP': 0, 'TP': 0},
 3: {'FN': 390, 'FP': 0, 'TP': 0},
 4: {'FN': 27, 'FP': 0, 'TP': 0},
 5: {'FN': 0, 'FP': 0, 'TP': 0},
 6: {'FN': 20, 'FP': 0, 'TP': 0},
 7: {'FN': 0, 'FP': 0, 'TP': 0},
 8: {'FN': 0, 'FP': 0, 'TP': 0}}

## Train and Evaluate for Multiple Epochs

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001)
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10
epoch = 0
print_freq = 20

In [ ]:
def train_eval(model, train_loader, test_loader, iou_thres=0.5, num_epochs=10):
    train_losses = []
    eval_threatscores = []
    eval_final_res = []
    best_eval_ts = 0

    best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        loss = train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq)
        train_losses.append(loss)
        
        final_res, final_TP, final_FP, final_FN, final_num_objects = evaluate_one_epoch(test_loader, iou_thres=0.5)

        print("epoch: {}".format(epoch))
        print(final_TP, final_FP, final_FN, final_num_objects)
        eval_final_res.append(final_res)
        eval_ts = evaluate_threst_score(final_TP, final_FP, final_FN)
        eval_threatscores.append(eval_ts)
        if epoch % 2 == 0:
            print("epoch: {} eval_ts {}".format(epoch, eval_ts))

        if eval_ts > best_eval_ts:
            best_eval_ts = eval_ts 
            best_model_wts = copy.deepcopy(model.state_dict())

    return model, best_model_wts, train_losses, eval_final_res

In [ ]:
model, best_model_wts, train_losses, eval_final_res = train_eval(model, train_loader,
                                                                 test_loader, iou_thres=0.5,
                                                                 num_epochs=10)

In [ ]:
eval_final_res

[{0: {'FN': 158, 'FP': 0, 'TP': 0},
  1: {'FN': 0, 'FP': 0, 'TP': 0},
  2: {'FN': 3073.0, 'FP': 12599.0, 'TP': 1.0},
  3: {'FN': 390, 'FP': 0, 'TP': 0},
  4: {'FN': 27, 'FP': 0, 'TP': 0},
  5: {'FN': 0, 'FP': 0, 'TP': 0},
  6: {'FN': 20, 'FP': 0, 'TP': 0},
  7: {'FN': 0, 'FP': 0, 'TP': 0},
  8: {'FN': 0, 'FP': 0, 'TP': 0}},
 {0: {'FN': 158, 'FP': 0, 'TP': 0},
  1: {'FN': 0, 'FP': 0, 'TP': 0},
  2: {'FN': 3070.0, 'FP': 846.0, 'TP': 4.0},
  3: {'FN': 390, 'FP': 0, 'TP': 0},
  4: {'FN': 27, 'FP': 0, 'TP': 0},
  5: {'FN': 0, 'FP': 0, 'TP': 0},
  6: {'FN': 20, 'FP': 0, 'TP': 0},
  7: {'FN': 0, 'FP': 0, 'TP': 0},
  8: {'FN': 0, 'FP': 0, 'TP': 0}}]

## Evaluate the trained model on the test set

## Customize Fast RCNN

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

#### 1. Mobilenet_v2

In [ ]:
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                    aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)
model = torchvision.models.detection.faster_rcnn.FasterRCNN(backbone,
                    num_classes=21,
                    rpn_anchor_generator=anchor_generator,
                    box_roi_pool=roi_pooler)

NameError: ignored

In [ ]:
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)

#### 2. CustomVGG16

In [ ]:
def customize_VGG16():
    model = torchvision.models.vgg16(pretrained=True)
    
    features = list(model.features)[:30]
    classifier = model.classifier
    
    classifier = list(classifier)
    # delete the Linear layer
    del classifier[6]
    classifier = nn.Sequential(*classifier)

    #freeze top4 conv layer
    for layer in features[:10]:
        for p in layer.parameters():
            p.requires_grad = False
    features = nn.Sequential(*features)
        
    return features, classifier
backbone, box_head = customize_VGG16()
backbone.out_channels = 512
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                           aspect_ratios=((0.5, 1.0, 2.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)
